## Margin Call and Liquidation Framework

In Lombard lending, the loan-to-value (LTV) ratio is the primary risk indicator, as it directly measures how well the outstanding loan is covered by the pledged collateral.

In this model, a margin call threshold is set at **75% LTV** and is intended as an early warning mechanism. When this level is breached, the borrower is expected to restore adequate collateral coverage, typically by posting additional collateral or partially reducing the loan exposure.

A second threshold at **90% LTV** represents the liquidation level. Breaching this threshold indicates a severe deterioration in collateral value, at which point the lender may initiate forced liquidation of part of the collateral portfolio in order to bring the loan back within acceptable risk limits.

In [ ]:
import pandas as pd
import numpy as np
initial_loan = 470000
price_matrix = pd.read_excel(r"price_matrix.xlsx", index_col='Date')

In [222]:
price_matrix = price_matrix.rename(columns={"Cash" : "Cash_Market_Value"})

def notional_asset (df):
    collateral_value_ptf_initial = 1000000
    allocation_equity = 0.50
    allocation_bond = 0.40
    allocation_cash = 0.05

    df['Equity_notional'] = ((collateral_value_ptf_initial * allocation_equity))
    df['Bond_notional'] = ((collateral_value_ptf_initial * allocation_bond))
    df['Cash_notional'] = ((collateral_value_ptf_initial * allocation_cash))

    df['Equity_notional'] = df['Equity_notional'].astype(int)
    df['Bond_notional'] = df['Bond_notional'].astype(int)
    df['Cash_notional'] = df['Cash_Market_Value'].astype(int)

    return df 

Final_matrix = notional_asset(price_matrix)

def asset_unit (df):

    df['Equity_unit'] = round((df['Equity_notional'].iloc[0] / df['Price_Equity'].iloc[0]))
    df['Bond_unit'] = round((df['Bond_notional'].iloc[0] / df['Price_Bond'].iloc[0]))

    return df

Final_matrix = asset_unit(Final_matrix )

def asset_market_value (df):

    df['Equity_MV'] = df['Equity_unit'] * df['Price_Equity']
    df['Bond_MV'] = df['Bond_unit'] * df['Price_Bond']
    df['Cash_MV'] = df['Cash_notional']

    return df

Final_matrix = asset_market_value(Final_matrix)

def apply_haircuts(df, haircut_map=None, default_haircut=0.35, value_suffix="_MV", out_suffix="_Eligible_Value"):
    if haircut_map is None:
        haircut_map = {"Equity": 0.30, "Bond": 0.15, "Cash": 0.05}

    df = df.copy()
    value_cols = [c for c in df.columns if c.endswith(value_suffix)]

    for col in value_cols:
        asset = col.replace(value_suffix, "")
        h = haircut_map.get(asset, default_haircut)
        df[f"{asset}{out_suffix}"] = df[col] * (1 - h)

    return df

Final_matrix = apply_haircuts(Final_matrix)

Final_matrix ['Tot_Eligible'] = (Final_matrix['Equity_Eligible_Value'] + Final_matrix['Bond_Eligible_Value'] + Final_matrix['Cash_Eligible_Value'])
Final_matrix ['Tot_Eligible'] = Final_matrix['Tot_Eligible'].astype(int)

#LTV
Final_matrix['LTV_daily'] = (initial_loan/Final_matrix ['Tot_Eligible'])
Final_matrix['% LTV_daily'] = round((initial_loan/Final_matrix ['Tot_Eligible']) * 100)

Final_matrix

,Price_Equity,Price_Bond,Cash_Market_Value,Equity_notional,Bond_notional,Cash_notional,Equity_unit,Bond_unit,Equity_MV,Bond_MV,Cash_MV,Equity_Eligible_Value,Bond_Eligible_Value,Cash_Eligible_Value,Tot_Eligible,LTV_daily,% LTV_daily
Date,,,,,,,,,,,,,,,,,
2020-01-02,55.02,5.19,50000,500000,400000,50000,9088,77071,500021.76,399998.49,50000,350015.232,339998.7165,47500.0,737513,0.637277,64.0
2020-01-03,54.97,5.20,50000,500000,400000,50000,9088,77071,499567.36,400769.20,50000,349697.152,340653.8200,47500.0,737850,0.636986,64.0
2020-01-06,54.85,5.20,50000,500000,400000,50000,9088,77071,498476.80,400769.20,50000,348933.760,340653.8200,47500.0,737087,0.637645,64.0
2020-01-07,55.29,5.20,50000,500000,400000,50000,9088,77071,502475.52,400769.20,50000,351732.864,340653.8200,47500.0,739886,0.635233,64.0
2020-01-08,55.53,5.19,50000,500000,400000,50000,9088,77071,504656.64,399998.49,50000,353259.648,339998.7165,47500.0,740758,0.634485,63.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-05,111.86,4.93,50000,500000,400000,50000,9088,77071,1016583.68,379960.03,50000,711608.576,322966.0255,47500.0,1082074,0.434351,43.0
2026-01-06,112.14,4.92,50000,500000,400000,50000,9088,77071,1019128.32,379189.32,50000,713389.824,322310.9220,47500.0,1083200,0.433900,43.0
2026-01-07,112.68,4.94,50000,500000,400000,50000,9088,77071,1024035.84,380730.74,50000,716825.088,323621.1290,47500.0,1087946,0.432007,43.0


In [216]:
notional_unit_matrix = Final_matrix.loc[:,['Equity_notional','Bond_notional', 'Cash_notional', 'Equity_unit', 'Bond_unit']]
LTV_Haircuts_matrix = Final_matrix.loc[:,['Equity_MV', 'Bond_MV', 'Cash_MV', 'Equity_Eligible_Value', 'Bond_Eligible_Value', 'Cash_Eligible_Value', 'Tot_Eligible', 'LTV_daily', '% LTV_daily' ]]
notional_unit_matrix.head()

,Equity_notional,Bond_notional,Cash_notional,Equity_unit,Bond_unit
Date,,,,,
2020-01-02,500000,400000,50000,9088,77071
2020-01-03,500000,400000,50000,9088,77071
2020-01-06,500000,400000,50000,9088,77071
2020-01-07,500000,400000,50000,9088,77071
2020-01-08,500000,400000,50000,9088,77071


In [217]:
LTV_Haircuts_matrix

,Equity_MV,Bond_MV,Cash_MV,Equity_Eligible_Value,Bond_Eligible_Value,Cash_Eligible_Value,Tot_Eligible,LTV_daily,% LTV_daily
Date,,,,,,,,,
2020-01-02,500021.76,399998.49,50000,350015.232,339998.7165,47500.0,737513,0.637277,64.0
2020-01-03,499567.36,400769.20,50000,349697.152,340653.8200,47500.0,737850,0.636986,64.0
2020-01-06,498476.80,400769.20,50000,348933.760,340653.8200,47500.0,737087,0.637645,64.0
2020-01-07,502475.52,400769.20,50000,351732.864,340653.8200,47500.0,739886,0.635233,64.0
2020-01-08,504656.64,399998.49,50000,353259.648,339998.7165,47500.0,740758,0.634485,63.0
...,...,...,...,...,...,...,...,...,...
2026-01-05,1016583.68,379960.03,50000,711608.576,322966.0255,47500.0,1082074,0.434351,43.0
2026-01-06,1019128.32,379189.32,50000,713389.824,322310.9220,47500.0,1083200,0.433900,43.0
2026-01-07,1024035.84,380730.74,50000,716825.088,323621.1290,47500.0,1087946,0.432007,43.0


###

Core of Lombard is the calculation of LTV. If the collateral value drops the LTV will increase increasing risk. in order to handle this a margin call is selected at 75%. Basically it means, whenever LTV will breach 75% (basically when collateral value decrease significantly), the borrower is required to add other collateral to keep lower LTV (around 64% defined at t0). for the same reason, in case of extreme shock, liquidation call is settled at LTV = 90%

In [224]:
margin_call = (0.75)
liquidation_call = (0.90)
# conditions
condition_margin_call = LTV_Haircuts_matrix['LTV_daily'] >= margin_call
condition_liquid_call = LTV_Haircuts_matrix['LTV_daily'] >= liquidation_call

def margin_liquidation_call (df, margin_condition, liq_condition):

    df['Margin_FLAG'] = np.where(margin_condition, "Margin Call", "No risks")
    df["Liquidation_FLAG"] = np.where(liq_condition, "Liquidation Call", "No risks")
    
    return df

LTV_Haircuts_matrix = margin_liquidation_call(LTV_Haircuts_matrix, condition_margin_call, condition_liquid_call)

LTV_Haircuts_matrix

,Equity_MV,Bond_MV,Cash_MV,Equity_Eligible_Value,Bond_Eligible_Value,Cash_Eligible_Value,Tot_Eligible,LTV_daily,% LTV_daily,Margin_FLAG,Liquidation_FLAG
Date,,,,,,,,,,,
2020-01-02,500021.76,399998.49,50000,350015.232,339998.7165,47500.0,737513,0.637277,64.0,No risks,No risks
2020-01-03,499567.36,400769.20,50000,349697.152,340653.8200,47500.0,737850,0.636986,64.0,No risks,No risks
2020-01-06,498476.80,400769.20,50000,348933.760,340653.8200,47500.0,737087,0.637645,64.0,No risks,No risks
2020-01-07,502475.52,400769.20,50000,351732.864,340653.8200,47500.0,739886,0.635233,64.0,No risks,No risks
2020-01-08,504656.64,399998.49,50000,353259.648,339998.7165,47500.0,740758,0.634485,63.0,No risks,No risks
...,...,...,...,...,...,...,...,...,...,...,...
2026-01-05,1016583.68,379960.03,50000,711608.576,322966.0255,47500.0,1082074,0.434351,43.0,No risks,No risks
2026-01-06,1019128.32,379189.32,50000,713389.824,322310.9220,47500.0,1083200,0.433900,43.0,No risks,No risks
2026-01-07,1024035.84,380730.74,50000,716825.088,323621.1290,47500.0,1087946,0.432007,43.0,No risks,No risks


In [219]:
LTV_Haircuts_matrix[LTV_Haircuts_matrix['Margin_FLAG'] == 'Margin Call']

,Equity_MV,Bond_MV,Cash_MV,Equity_Eligible_Value,Bond_Eligible_Value,Cash_Eligible_Value,Tot_Eligible,LTV_daily,% LTV_daily,Margin_FLAG,Liquidation_FLAG
Date,,,,,,,,,,,


In [220]:
LTV_Haircuts_matrix[LTV_Haircuts_matrix['LTV_daily'] > 0.75]

,Equity_MV,Bond_MV,Cash_MV,Equity_Eligible_Value,Bond_Eligible_Value,Cash_Eligible_Value,Tot_Eligible,LTV_daily,% LTV_daily,Margin_FLAG,Liquidation_FLAG
Date,,,,,,,,,,,


## Results and Interpretation

Under normal market conditions, the Lombard position remains comfortably within margin call limits. This reflects a conservative initial structuring of the loan and a well-diversified collateral portfolio.

Risk escalation does not materialise in the baseline scenario and only emerges under severe stress conditions. This behaviour is consistent with the intended role of Lombard lending as a resilient financing tool for high-net-worth clients rather than a short-term trading facility.

Despite the severe equity market drawdown observed during the COVID-19 crisis, the Lombard position does not breach margin call thresholds under the baseline configuration, highlighting the stabilising role of diversified collateral, conservative haircuts and cash buffers.

During the COVID-19 market dislocation, the Lombard position approaches the margin call threshold, with a peak LTV of approximately 74%. Despite the severity of the equity drawdown, margin call levels are not breached, highlighting the effectiveness of conservative haircuts and collateral diversification.